In [1]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [2]:
# get image from test-set-uncategorized.json (training set)
labels = os.path.join('test-set-uncategorized.json')
count = 1
phase_count = {
    "train": 1,
    "test": 1
}
phase = "train"
# degs =  [90,180,270]
degs = [90,180,270,360,450,540,630]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "mixed-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            phase_count[phase] += 1
            count += 1
            for deg in degs:
                name = "mixed-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w64_033.jpg
mixed-train-0000000001
2 90
3 180
4 270
5 450
6 540
7 630
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/7b_067.JPG
mixed-train-0000000008
9 90
10 180
11 270
12 360
13 450
14 540
15 630
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_064.jpg
mixed-train-0000000016
17 90
18 180
19 270
20 360
21 450
22 540
23 630
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_070.jpg
mixed-train-0000000024
25 90
26 180
27 270
28 360
29 450
30 540
31 630
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_058.jpg
mixed-train-0000000032
33 90
34 180
35 270
36 360
37 450
38 540
39 630
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_040.JPG
mixed-train-0000000040
7 https://raw.githubusercontent.com/RT

308 90
309 180
310 270
311 360
312 450
313 540
314 630
53 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_080.JPG
mixed-train-0000000315
316 90
317 180
318 270
319 360
320 450
321 540
322 630
54 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_041.jpg
mixed-train-0000000323
324 90
325 180
326 270
327 360
328 450
329 540
330 630
55 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w31_023.jpg
mixed-train-0000000331
332 90
333 180
334 270
335 360
336 450
337 540
338 630
56 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/6a_004.jpg
mixed-train-0000000339
57 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/6a_010.jpg
mixed-train-0000000340
341 90
342 270
343 360
344 540
345 630
58 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_055.jpg
mixed-train-0000000346


104 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_116.jpg
mixed-train-0000000614
615 90
616 180
617 270
618 360
619 450
620 540
621 630
105 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_117.jpg
mixed-train-0000000622
623 90
624 180
625 270
626 360
627 450
628 540
629 630
106 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_084.JPG
mixed-train-0000000630
631 90
632 180
633 270
634 360
635 450
636 540
637 630
107 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_090.jpg
mixed-train-0000000638
639 90
640 180
641 270
642 450
643 540
644 630
108 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w31_033.jpg
mixed-train-0000000645
646 90
647 180
648 270
649 360
650 450
651 540
652 630
109 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_051.jpg
mixed-tr

949 90
950 180
951 270
952 360
953 450
954 540
955 630
152 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/2a_047.JPG
mixed-train-0000000956
957 90
958 180
959 270
960 360
961 450
962 540
963 630
153 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_034.jpg
mixed-train-0000000964
965 90
966 180
967 270
968 360
969 450
970 540
971 630
154 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_008.jpg
mixed-train-0000000972
973 90
974 180
975 270
976 360
977 450
978 540
979 630
155 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_036.JPG
mixed-train-0000000980
156 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_022.JPG
mixed-train-0000000981
982 90
983 180
984 270
985 360
986 450
987 540
988 630
157 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/15b_006.jpg
m

1284 90
1285 180
1286 360
1287 450
1288 540
1289 630
199 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_029.jpg
mixed-train-0000001290
1291 90
1292 180
1293 270
1294 360
1295 450
1296 540
1297 630
200 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/4a_017.JPG
mixed-train-0000001298
1299 90
1300 180
1301 270
1302 360
1303 450
1304 540
1305 630
201 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w32_017.jpg
mixed-train-0000001306
1307 180
1308 270
1309 360
1310 450
1311 630
202 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/7b_024.JPG
mixed-train-0000001312
1313 90
1314 270
1315 360
1316 630
203 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/7b_030.JPG
mixed-train-0000001317
1318 90
1319 180
1320 270
1321 360
1322 450
1323 540
1324 630
204 https://raw.githubusercontent.com/RTANC/malaria_dataset

1602 90
1603 180
1604 270
1605 360
1606 450
1607 540
1608 630
247 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/6a_008.jpg
mixed-train-0000001609
248 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/2a_016.JPG
mixed-train-0000001610
249 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/2a_017.JPG
mixed-train-0000001611
250 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/6a_009.jpg
mixed-train-0000001612
1613 90
1614 180
1615 270
1616 360
1617 450
1618 540
1619 630
251 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w66_070.jpg
mixed-train-0000001620
1621 90
1622 180
1623 270
1624 360
1625 450
1626 540
1627 630
252 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/test-set-uncategorized/w31_006.jpg
mixed-train-0000001628
1629 90
1630 270
1631 360
1632 450
1633 540
1634 630
253 https://raw

In [3]:
print(phase_count['train'])

1909
